Entity Resolution
=========

This provides an overview of why and how Entity Resolution is becoming an important 
discipline in [Computer Science and Data Science](http://www.datacommunitydc.org/blog/2013/08/entity-resolution-for-big-data). This notebook explores why we need entity resolution and how to do it. Brief explanations
are given regarding commerical options. Open source options are also discussed. Some of these open
source options are demonstrated using Python.

## Why do we need entity resolution?

When doing a statistical analysis, you need to first identify your units of analysis. 
Borrowing from Allison ([Multiple Regression: A Primer](http://www.amazon.com/Multiple-Regression-Research-Methods-Statistics/dp/0761985336), p. 7):

> To do a regression analysis, you first need a set of cases (also called units of analysis or observations).
> In the social sciences, the cases are most often persons, but they could also be organizations, countries, or
> other groups. In economics, the cases are sometimes units of time, like years or quarters. For each case, you
> need measurements on all of the variables in the regession equation. 

Often, our data does not come to us in one table; ready for analysis. In this situation, we also need to be
aware of database concepts.In web development, we can say that we're concerned with defining the database 
model (see the Python web framework Django for [their model specifications](https://docs.djangoproject.com/en/1.8/topics/db/models/)). In the case of Entity Resolution, we are concerned with two concepts:

1. Primary Keys
1. Foreign Keys

In Django, each model requires exactly one field to have a primary key. A foreign key specifies a one-to-many 
relationship in Django. They have separate field types for one-to-one and many-to-many relationships between models. For our purposes we will exclude many-to-many relationships and include one-to-one relationships in our discussion of the Foreign key. The following is a demonstration from the [Django Documenation of the Foreign key](https://docs.djangoproject.com/en/1.8/ref/models/fields/#django.db.models.ForeignKey). 

``` Python
from django.db import models

class Car(models.Model):
    manufacturer = models.ForeignKey('Manufacturer')
    # ...

class Manufacturer(models.Model):
    # ...
    pass
```

We can see that each type of car could have multiple manufacturers. Several car manufacturers make mid-size sedans. Those manufacturers are reflected in the above Django model. The database models are not specific to Python but it does help to have concrete examples with easily accessible documentation.

When conducting a statistical analysis, we want to run our model on one table. Before conducting the analysis, we must be sure of the following:

1. Each unit of analysis occurs only once 
  * Primary keys must be present and unique
1. Each unit of analysis includes variables that have been correctly mapped from their original tables
  * Foreign keys must be present

Entity Resolution is a tool to define primary and foreign keys when these relationships are not previously well defined. For example, let's say people start using the web application described above. They populate the models with cars and
the manufacturers who produce them. The developers did not include standard names for cars and manufacturers because
they were unsure of what the future may hold. As the data scientist arrives on scene, they find that people have spelled
the manufacturer "GMC" in a variety of ways like "General Motors Corporation" and "General Motors". People have also 
mispelled the manufacturer name. Actually, they have done this across much of the manufacturer and car information. 

Given the state of this manufacturer/car data, the data scientist is no longer able to ascertain their unit of analysis. They must recreate primary and foreign keys before they can perform their statistical analysis. 

  
**TL;DR** We can't begin our analysis unless we have defined and identified quantifiable units of analysis. Data often shows up in multiple tables, we need to know about primary and foreign keys. We want to run our statistical analysis on a single table. We must determine primary and foreign keys before we can manipulate our data to perform the statistical analysis. Entity resolution can be used to establish primary and foreign keys when those relationships are not previously well defined.




## How can we accomplish entity resolution?

We have established that entity resolution is a way for the data scientist to restablish primary and foreign keys once people have inputted information which makes those relationships ambiguous from a statistical analysis standpoint. Here is the same goal restated by the [Stanford Entity Resolution Framework (SERF)](http://infolab.stanford.edu/serf/)Project.

> The goal of the SERF project is to develop a generic infrastructure for Entity Resolution (ER). ER (also known as
> deduplication, or record linkage) is an important information integration problem: The same "real-world entities"
> (e.g., customers, or products) are referred to in different ways in multiple data records. For instance, two records
> on the same person may provide different name spellings, and addresses may differ. The goal of ER is to "resolve"
> entities, by identifying the records that represent the same entity and reconciling them to obtain one record per
> entity.

You can accomplish entity resolution by:

1. Hiring a company
1. Doing it yourself
1. A little of both

We will explore each option. 

### Hire a company

[Basis Technology](http://www.basistech.com/) has a product called the [Rosette Entity Resolver](http://www.basistech.com/text-analytics/rosette/entity-resolver/). Their technology has been used
by "Amazon.com, EMC, Endeca/Oracle, Exalead/Dassault, Fujitsu, Google, Hewlett-Packard, Microsoft, Oracle, and governments around the world". If you're a newer smaller company then you might want to check out their [startup program](http://www.basistech.com/about/startup/).

### Do it yourself

There are number of open source tools and frameworks that I've found. None of them are an Apache projects. Here's what I've found:

1. [Dedupe](https://github.com/datamade/dedupe) python package
1. [Duke](https://github.com/larsga/Duke)
1. [elasticsearch-entity-resolution](https://github.com/YannBrrd/elasticsearch-entity-resolution)
1. [Berkeley Entity Resolution](https://github.com/gregdurrett/berkeley-entity)
1. [SERF Project](http://infolab.stanford.edu/serf/)
1. [Ch. 3 of Mining Massive Datasets](http://infolab.stanford.edu/~ullman/mmds/ch3.pdf)

Of these options, Dedupe is the only one maintained by an organization ([Datamade](http://datamade.us/)). My personal experience with Duke is that it seemed to have some buzz but none of the demo examples worked for me. The elasticsearch entity resolution plugin is based on Duke (haven't personally tried it). The Berkeley ER project looks like it's Greg's thesis and appears to be a work in progress. When I contacted one of the researchers at the SERF project, they said it's no longer active. Ch.3 of Mining massive datasets included an entity resolution example that may work for certain use cases.

Let's take a closer look at *Dedupe* and Ch.3 of Mining Massive datasets (hereafter referred to as *MMDS*). See **appendix 1 & 2** for examples.

### Do some of it yourself but hire a company

One approach is to refurbish your dataset by kicking the complexity to some else's API. Depending on your
data, you may be able to take advantage of the [Google Places API](https://developers.google.com/places/). This would allow you to return standard names for locations and organizations; you may even be able to cross-reference their phone numbers. This API is considerably different than the one produced by Facebook or Foursquare because it's not user-generated; from what I can tell. See **Appendix 3** for an example.

## Appendix 0: Getting Data

Our data is from the Dedupe [test datasets](https://github.com/datamade/dedupe/tree/master/tests/datasets).

In [1]:
import pandas as pd

#restaurant_1 = pd.read_csv("datasets/restaurant-1.csv")
#restaurant_2 = pd.read_csv("datasets/restaurant-2.csv")

#print("Columns are identical: {}".format(restaurant_1.columns == restaurant_2.columns))

In [2]:
#restaurant_1.describe()

## Appendix 1: MMDS Approach

The main moving part here is the edit distance. 

## Appendix 2: Dedupe (Python Package)

Note that [dedupe tests](https://github.com/datamade/dedupe/tree/master/tests) includes the referenced datasets. I've modified the tests so they play well within a notebook. We will use Dedupe to solve the following problems:


1. Deduplication
1. Record linkage

See [dedupe examples](https://github.com/datamade/dedupe-examples) for examples that include mysql and postgres. The examples that I've demonstrated below are taken from these scripts. I've modified the examples to exclude training the data; so you can just run this notebook and see results.

### Deduplication

In [3]:
# filepaths

input_file = 'csv_example/csv_example_messy_input.csv'
output_file = 'csv_example/csv_example_output.csv'
settings_file = 'csv_example/csv_example_learned_settings'
training_file = 'csv_example/csv_example_training.json'


In [4]:
import os
import csv
import re
import collections

import optparse
from numpy import nan

import dedupe
from unidecode import unidecode


In [5]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    import unidecode
    #column = column.decode("utf8")
    column = unidecode.unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column :
        column = None
    return column

def readData(filename):
    """
    Read in our data from a CSV file and create a dictionary of records, 
    where the key is a unique record ID and each value is dict
    """

    data_d = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)

    return data_d


In [6]:
# run deduplication

print('importing data ...')
data_d = readData(input_file)

# ## Training

if os.path.exists(settings_file):
    # we already know the settings_file has been created
    print('reading from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)
        
else:
    raise FileNotFoundError("Contact notebook author")

importing data ...
reading from csv_example/csv_example_learned_settings


In [11]:
# ## Blocking

print('blocking...')

# ## Clustering

# Find the threshold that will maximize a weighted average of our precision and recall. 
# When we set the recall weight to 2, we are saying we care twice as much
# about recall as we do precision.
#
# If we had more data, we would not pass in all the blocked data into
# this function but a representative sample.
threshold = deduper.threshold(data_d, recall_weight=2)


# `match` will return sets of record IDs that dedupe
# believes are all referring to the same entity.

print('clustering...')
#clustered_dupes = deduper.match(data_d, threshold)

print('# duplicate sets', len(clustered_dupes))
    




INFO:dedupe.blocking:Canopy: TfidfTextCanopyPredicate: (0.6, Address)
INFO:dedupe.blocking:Canopy: TfidfNGramCanopyPredicate: (0.8, Address)
INFO:dedupe.blocking:Canopy: TfidfTextCanopyPredicate: (0.6, Site name)
INFO:dedupe.api:0 blocks


blocking...


INFO:dedupe.api:Maximum expected recall and precision
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of

clustering...
# duplicate sets 795


In [8]:
# ## Writing Results

# Write our original data back out to a CSV with a new column called 
# 'Cluster ID' which indicates which records refer to each other.

cluster_membership = {}
cluster_id = 0
for (cluster_id, cluster) in enumerate(clustered_dupes):
    id_set, scores = cluster
    cluster_d = [data_d[c] for c in id_set]
    canonical_rep = dedupe.canonicalize(cluster_d)
    for record_id, score in zip(id_set, scores) :
        cluster_membership[record_id] = {
            "cluster id" : cluster_id,
            "canonical representation" : canonical_rep,
            "confidence": score
        }

singleton_id = cluster_id + 1

with open(output_file, 'w') as f_output:
    writer = csv.writer(f_output)

    with open(input_file) as f_input :
        reader = csv.reader(f_input)

        heading_row = next(reader)
        heading_row.insert(0, 'confidence_score')
        heading_row.insert(0, 'Cluster ID')
        canonical_keys = canonical_rep.keys()
        for key in canonical_keys:
            heading_row.append('canonical_' + key)
        
        writer.writerow(heading_row)

        for row in reader:
            row_id = int(row[0])
            if row_id in cluster_membership :
                cluster_id = cluster_membership[row_id]["cluster id"]
                canonical_rep = cluster_membership[row_id]["canonical representation"]
                row.insert(0, cluster_membership[row_id]['confidence'])
                row.insert(0, cluster_id)
                for key in canonical_keys:
                    row.append(canonical_rep[key].encode('utf8'))
            else:
                row.insert(0, None)
                row.insert(0, singleton_id)
                singleton_id += 1
                for key in canonical_keys:
                    row.append(None)
            writer.writerow(row)

Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'affinegap.affinegap.normalizedAffineGapDistance'
TypeError: object of type 'NoneType' has no len()
Exception ignored in: 'a

Given that this is example data, we already know which duplicates exist in the
date. Let's evaluate our procedure

In [9]:
# Evaluating output

manual_clusters = 'csv_example/csv_example_input_with_true_ids.csv'
dedupe_clusters = 'csv_example/csv_example_output.csv'

from csv_example.csv_evaluation import dupePairs, evaluateDuplicates

In [10]:
true_dupes = dupePairs(manual_clusters, 'True Id')
test_dupes = dupePairs(dedupe_clusters, 'Cluster ID')

evaluateDuplicates(test_dupes, true_dupes)

found duplicate
5628
precision
0.9774342572850035
recall
0.8324757869249395


### Record linkage

## Appendix 3: Google Places API

Please verify with Google that this is not a violation of their terms of service before implementing this approach.
